#Select VASCA sources based on chosen criteria

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_dir_name = "./vasca_pipeline/"+region_name+"/"
region_cat_fname = region_dir_name+"region_"+region_name+"_cat.fits"

### Prepare tt_sources with variables for selection

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from vasca.region import Region
import vasca.visualization as vvis
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, get_col_cycler,add_ogrp, color_palette, nb_fig
from astropy import units as uu
from astropy.table import Column

# Get region
rc = Region()
rc.load_from_fits(region_cat_fname)
rc.tt_sources.sort(["dec"])

#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp","match_distance","sp_type","main_id"],"simbad_match_id",select_matched=True)
rc.tt_sources.rename_column("match_distance","match_distance_simbad")
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
sel_simbad = rc.tt_sources["sel"].data
print(f"Number of SIMBAD matches is: {sel_simbad.sum()}  ({100* sel_simbad.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiadr3",["PQSO", "PGal", "PSS","match_distance","Gmag_abs","BP-RP", "ogrp", "Plx","e_Plx","Plx_dist","VarFlag","RPlx"],"gaiadr3_match_id",select_matched=True)
rc.tt_sources.rename_column("match_distance","match_distance_gaiadr3")
rc.tt_sources.rename_column("ogrp","ogrp_gaiadr3")
sel_gaiadr3 = rc.tt_sources["sel"].data
print(f"Number of GAIA matches is: {sel_gaiadr3.sum()}  ({100*sel_gaiadr3.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiaedr3_wd",["Pwd"],"rg_src_id",select_matched=True)
sel_gaia2wd = rc.tt_sources["sel"].data
print(f"Number of GAIA WD matches is: {sel_gaia2wd.sum()}  ({100*sel_gaia2wd.sum() / len(rc.tt_sources)} )%")

sel_match = np.array(sel_simbad+sel_gaiadr3+sel_gaia2wd, dtype=bool)
print(f"Total number of matches is: {sel_match.sum()}  ({100*sel_match.sum() / len(rc.tt_sources)} )%")

#Copy lomb scargle result into tt_sources
rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_model_rchiq"],"rg_src_id")

rc.tt_sources["sel"]=True
print(f"Total number of sources is: {rc.tt_sources['sel'].sum()}")

display(rc.tt_simbad[rc.tt_simbad["ogrp"]=="WD*"])
display(rc.tt_gaiadr3)

Number of SIMBAD matches is: 2687  (63.945740123750596 )%


Number of GAIA matches is: 3302  (78.58162779628748 )%
Number of GAIA WD matches is: 11  (0.2617801047120419 )%
Total number of matches is: 3656  (87.00618752974773 )%
Total number of sources is: 4202


main_id,ra,dec,ra_prec,dec_prec,coo_err_maja,coo_err_mina,coo_err_angle,coo_qual,coo_wavelength,coo_bibcode,otype,otypes,distance_distance,distance_q,distance_unit,distance_merr,distance_perr,distance_method,distance_bibcode,match_distance,rvz_type,rvz_radvel,rvz_error,rvz_qual,rvz_wavelength,rvz_bibcode,z_value,sp_type,sp_qual,sp_bibcode,script_number_id,rg_src_id,simbad_match_id,ogrp
,deg,deg,,,mas,mas,deg,,,,,,,,,,,,,arcsec,,km / s,km / s,,,,,,,,,,,
bytes32,float64,float64,int16,int16,float32,float32,int16,bytes1,bytes1,bytes32,bytes32,bytes32,float64,bytes1,bytes4,float64,float64,bytes8,bytes19,float32,bytes1,float64,float32,bytes1,bytes1,bytes32,float64,bytes32,bytes32,bytes32,int32,int32,int32,bytes8


_q,ra,dec,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,PQSO,PGal,PSS,RPlx,VarFlag,o_Gmag,RFRP,RFBP,AG,E_BP-RP,rg_src_id,match_distance,gaiadr3_match_id,Gmag_abs,Plx_dist,ogrp
,deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,,,,,,,,mag,mag,,arcsec,,,pc,
int32,float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,bytes12,bytes15,bytes19,int32,bytes10,bytes16,bytes17,bytes21,float64,float64,float64,float64,float64,float64,bytes13,int16,float32,float32,float32,float32,int32,float64,int32,float32,float64,bytes8
1,150.2,2.1,3836069006998531584,2.2872,3.3137,--,--,--,--,--,--,--,--,95.82888,1.689,20.733625,93.75,16.38,20.408556,130.9,9.985,19.455320,0.953236,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110481501947501974,1237651753997304055,--,--,--,J100046.74+020404.4,--,N6SS000981,--,--,--,150.2,2.1,0.001869,0.000724,0.996765,--,NOT_AVAILABLE,79,13.11,5.725,--,--,41,0.6956587221689177,0,-100.0,--,none
2,150.1,2.0,3836046123412862208,1.5776,2.0453,--,--,--,--,--,--,--,--,90.12499,2.676,20.800253,37.12,20.52,21.414429,59.56,10.83,20.310590,1.103838,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110451501456232172,1237651736280760581,--,--,--,J100034.94+020234.9,--,N6SS004829,--,--,--,150.1,2.0,0.001265,0.000013,0.997987,--,NOT_AVAILABLE,43,5.497,1.809,--,--,68,0.6420858569707824,1,-100.0,--,none
4,150.1,2.1,3836257710681556224,0.5228,0.5439,-0.0737,0.5331,0.702,-0.580,0.607,0.396,0.641,1.065,231.69009,2.456,19.775099,160.4,10.57,19.825287,132.1,14.81,19.445590,0.379698,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,110521501020297033,1237651753997238486,--,--,--,J100024.44+020619.5,--,N6SV003245,--,--,--,150.1,2.1,1.000000,0.000000,0.000000,-0.1383,VARIABLE,182,8.917,15.18,--,--,87,1.015315761207988,2,-100.0,--,AGN
7,215.1,52.6,1607957522794289024,0.1805,0.2101,-0.4327,0.2279,0.356,0.190,0.223,0.302,0.256,1.036,311.79894,1.36,19.452680,178.4,9.809,19.709816,189.3,7.18,19.055190,0.654627,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171142150651746989,1237661385994862739,--,--,--,J142015.62+523718.5,--,N5MV001218,--,--,--,215.1,52.6,1.000000,0.000000,0.000000,-1.8990,VARIABLE,482,26.36,18.19,--,--,302,0.26795579682948123,3,-100.0,--,AGN
8,215.0,52.7,1607958480571152256,0.8988,0.7715,-1.1739,0.6843,0.532,-0.092,1.013,0.524,0.894,1.079,127.05163,2.726,20.427416,98.51,10.98,20.354832,127.2,7.357,19.486479,0.868353,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171212150426430532,1237661385994862858,--,--,--,J142010.22+524029.6,--,N5MV001750,--,--,--,215.0,52.7,0.999765,0.000220,0.000015,-1.7155,VARIABLE,283,17.29,8.972,--,--,314,0.5654019830257911,4,-100.0,--,AGN
9,214.9,52.5,1607951746062355328,0.2074,0.2337,0.1023,0.2656,0.429,-0.239,0.256,-0.357,0.272,0.966,233.47251,0.8803,19.766777,140.9,7.083,19.965982,148.7,6.695,19.317305,0.648678,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171052149430476033,1237661385994862854,--,--,--,J141946.35+523246.3,--,N5MV000642,--,--,--,214.9,52.5,1.000000,0.000000,0.000000,0.3850,VARIABLE,523,22.21,19.9,--,--,336,0.5780435001189599,5,-100.0,9775.1711,AGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### Print info for selected sources

In [5]:
# Make selection
sel_otype = rc.tt_sources["otype"] ==  "PM*" #"SN*" #"RR*"# "No*"#"HS?" #"BS*" # # "SN*" #"EB*"
sel_otypes =   (rc.tt_sources["Pwd"] >0.9) # (rc.tt_sources["otype"] ==  "WD?")  + #  + (rc.tt_sources["RPlx"]>4) (rc.tt_sources["otype"] =="WD*")*(rc.tt_sources["sp_type"] !="none")+
sel_flux_nxv = rc.tt_sources["flux_nxv"][:,0]>2
sel_flux_hr = rc.tt_sources["hr"]>2
sel_src_id = rc.tt_sources["rg_src_id"]==15179
sel_period = (rc.tt_sources["ls_peak_pval"]>-0.5) * (rc.tt_sources["ls_peak_pval"]<0.000000573303) * (rc.tt_sources["ls_model_rchiq"]<1.1)# # 3 sigma 0.002699796063, 4 sigma 0.000063342484, 5 sigma 0.000000573303
sel_srcs =  sel_otype #sel_period #sel_otypes  #sel_src_id  #sel_flux_nxv #sel_otype # sel_flux_nxv_fuv # sel_flux_hr#

#Print oout list of source IDs to be used for further use, e.g. in inspect_sources.ipynb
print("\nsrcs_ids = [", end="")
for src_id in rc.tt_sources[sel_srcs]["rg_src_id"].data:
    print(src_id,",", end="")
print("]\n")
print("Total Nr. of sources:",sel_srcs.sum(),"\n")

#Display match and simbad table for selected sources
cols_write = ["src_name","ra","dec","ogrp_gaiadr3","otype","sp_type","main_id","nr_det","flux","flux_cpval","flux_ne","Plx_dist","ls_peak_pval","ls_model_rchiq","VarFlag","Pwd","RPlx","rg_src_id"]
display(rc.tt_sources[sel_srcs][cols_write] )#["sp_type","main_id"]
rc.tt_sources[sel_srcs][cols_write].write(region_dir_name+"sources.ecsv",overwrite=True,delimiter=',')

col_rd = ["Pwd","Plx_dist","RPlx","flux_ne"]
for col in col_rd:
    rc.tt_sources[col] = np.round(rc.tt_sources[col],2)
cols_tex = ["src_name","main_id","sp_type"]+col_rd+ ["flux_cpval"]
rc.tt_sources[sel_srcs][cols_tex].write(region_dir_name+"sources.tex", overwrite=True)



srcs_ids = [1521282 ,309137 ,214046 ,570092 ,70723 ,1747015 ,242634 ,1469861 ,1775540 ,1570009 ,1584655 ,192348 ,866068 ,160031 ,883278 ,805990 ,821390 ,221953 ,653072 ,124748 ,]

Total Nr. of sources: 20 



src_name,ra,dec,ogrp_gaiadr3,otype,sp_type,main_id,nr_det,flux,flux_cpval,flux_ne,Plx_dist,ls_peak_pval,ls_model_rchiq,VarFlag,Pwd,RPlx,rg_src_id
,deg,deg,,,,,,1e-06 Jy,,,,,,,,,
bytes24,float64,float64,bytes8,bytes32,bytes32,bytes32,int32[2],float32[2],float32[2],float32[2],float32,float32,float32,bytes32,float32,float32,int32
1UVA J64716.1-361135.0,101.81710210989921,-36.193058721758696,Star*,PM*,,CD-36 3152,17 .. -1,538.3593 .. -1.0,0.0 .. -1.0,0.04 .. --,478.86,-1.0,-1.0,NOT_AVAILABLE,-1.0,164.21,1521282
1UVA J24936.5-244414.0,42.4020294028045,-24.7372359958571,none,PM*,M0.5,LP 830-44,19 .. 2,7.402895 .. 2.1757395,0.0 .. 0.85363716,0.42 .. --,-1.0,-1.0,-1.0,none,-1.0,-1.0,309137
1UVA J110206.5-225229.4,165.52697545737453,-22.87482653838341,none,PM*,K5Ve,RX J1102.1-2252,20 .. 20,79.83186 .. 9.882956,0.0 .. 0.0,0.67 .. 1.44,56.52,-1.0,-1.0,NOT_AVAILABLE,-1.0,875.09,214046
1UVA J194443.4-74959.3,296.18095254478396,-7.833144728280071,none,PM*,,GUVV J194443.4-074959.5,14 .. -1,26.761148 .. -1.0,0.0 .. -1.0,0.27 .. --,55.04,-1.0,-1.0,NOT_AVAILABLE,-1.0,225.17,570092
1UVA J22635.2-50518.1,36.64679343377384,-5.088367301961614,none,PM*,,2XLSSd J022635.2-050518,7 .. 1,3.4287777 .. 12.545136,0.0 .. -1.0,3.55 .. --,118.32,-1.0,-1.0,NOT_AVAILABLE,-1.0,218.0,70723
1UVA J21829.9-45513.5,34.62451156641868,-4.920414447220848,none,PM*,M1,2MASS J02182991-0455137,-1 .. 16,-1.0 .. 2.8671021,-1.0 .. 5.8832773e-25,-- .. 1.06,91.02,-1.0,-1.0,NOT_AVAILABLE,-1.0,588.8,1747015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1UVA J143735.2+345907.6,219.39646966499114,34.98545123380675,Star*,PM*,,2MASS J14373514+3459077,5 .. -1,2.8333015 .. -1.0,4.601553e-11 .. -1.0,0.88 .. --,130.16,-1.0,-1.0,NOT_AVAILABLE,-1.0,41.48,883278


In [4]:
srcs_ids = [357455 ,94932]